In [12]:
import torch

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Custom CNN Network using VGG Blocks

In [13]:
import torch
import torch.nn as nn

class load_customVGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(load_customVGG16, self).__init__()
        
        # Define the feature extractor part of VGG16
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Define the classifier part of VGG16
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Data Loader

In [15]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = 'ImageNet/'
def get_data_loaders(data_dir, batch_size):
    train_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=val_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return train_loader, val_loader

## Optimizers

In [16]:
import torch.optim as optim

# Function to create and return the model and optimizer
def create_model_optimizer(optimizer_name, lr):
    model = load_customVGG16(num_classes=10).to(device)
    optimizers = {
        'SGD': optim.SGD(model.parameters(), lr=lr, momentum=0.9),
        'Adam': optim.Adam(model.parameters(), lr=lr),
        'RMSprop': optim.RMSprop(model.parameters(), lr=lr),
        'AdamW': optim.AdamW(model.parameters(), lr=lr),
        'Adamax': optim.Adamax(model.parameters(), lr=lr),
        'SparseAdam': optim.SparseAdam(model.parameters(), lr=lr),
        'RAdam': optim.RAdam(model.parameters(), lr=lr),
        'NAdam': optim.NAdam(model.parameters(), lr=lr)
    }
    optimizer = optimizers[optimizer_name]
    return model, optimizer

# Training Loop

In [17]:
import pandas as pd
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

# Function to train and evaluate the model
def train_and_evaluate(optimizer_name, lr, train_loader, val_loader):
    model, optimizer = create_model_optimizer(optimizer_name, lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    scaler = GradScaler()

    best_val_acc = 0
    best_model_wts = None

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    # Initialize lists to store epoch-wise data
    epoch_data = {
        'Epoch': [],
        'Train Loss': [],
        'Train Acc': [],
        'Val Loss': [],
        'Val Acc': []
    }

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        loop = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{num_epochs}]', leave=True)
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loop.set_postfix(loss=loss.item())
        
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        val_loss, val_correct, val_total = 0.0, 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        scheduler.step()

        # Add epoch-wise data to the dictionary
        epoch_data['Epoch'].append(epoch + 1)
        epoch_data['Train Loss'].append(train_loss)
        epoch_data['Train Acc'].append(train_accuracy)
        epoch_data['Val Loss'].append(val_loss)
        epoch_data['Val Acc'].append(val_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}] - '
          f'Train Loss: {train_loss:.4f}, '
          f'Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, '
          f'Val Acc: {val_accuracy:.2f}%')

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_model_wts = model.state_dict().copy()
      
    print(f'Best Validation Accuracy: {best_val_acc:.2f}%')
    if best_model_wts:
        model.load_state_dict(best_model_wts)
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val_acc': best_val_acc
    }, f'Checkpoints_customVGG16/model_{optimizer_name}_batch_size_{batch_size}.pth')

    # Convert epoch-wise data dictionary to DataFrame
    df = pd.DataFrame(epoch_data)

    # Save DataFrame to CSV file
    df.to_csv(f'epoch_records__customVGG16/epoch_data_{optimizer_name}_batch_size_{batch_size}.csv', index=False)

    return train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
# List of optimizers to be used
optimizers_list = ['Adam', 'SGD', 'RMSprop', 'AdamW', 'Adamax', 'RAdam', 'NAdam']
# optimizers_list = ['SparseAdam']

# List of batch sizes to be used
batch_sizes = [8, 16, 32]
lr = 0.0001
num_epochs = 10

# Train and evaluate for different optimizers and batch sizes
loss_dict, acc_dict = {}, {}

for optimizer_name in optimizers_list:
    for batch_size in batch_sizes:
        print(f'Training with optimizer {optimizer_name} and batch size {batch_size}')
        train_loader, val_loader = get_data_loaders(data_dir, batch_size)
        train_losses, val_losses, train_accuracies, val_accuracies = train_and_evaluate(optimizer_name, lr, train_loader, val_loader)
        key = f'{optimizer_name}_bs{batch_size}'
        loss_dict[key] = {'train': train_losses, 'val': val_losses}
        acc_dict[key] = {'train': train_accuracies, 'val': val_accuracies}